## PRACTICA GUIADA

### PARTE I: Limpieza y transformación de datos

Esta práctica se propone brindar un catálogo de métodos y funciones en Pandas y Pyhton que podrán ser útiles a la hora de encarar tareas de limpieza de datos. 

En general, podemos identificar seis tipos de tareas u operaciones que aplicamos a los datos en la etapa de limpieza.

1. Estandarización de categorías (homogeneización)
2. Resolución de problemas de formato
3. Asignación de formatos adecuados (dtype)
4. Corrección de valores erróneos
5. Completar datos faltantes (missing data imputation)
6. Organización correcta del dataset (tidy data)

Las funciones y métodos presentados abarcan una o varias de estas operaciones.

### Remover duplicados

In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


* `duplicated()` devuelve un booleano identificando los casos duplicados.
* `drop_duplicates()` devuelve el `DataFrame` sin los casos duplicados

In [21]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [22]:
# Podemos definir algunos parámetros:

data.duplicated(['k1'],keep='last')

0     True
1     True
2    False
3     True
4     True
5     True
6    False
dtype: bool

In [23]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [24]:
# ~ not negacion
data[~data.duplicated()] == data.drop_duplicates()

,k1,k2
0,True,True
2,True,True
3,True,True
5,True,True


* Se puede utilizar `drop_duplicates()` para eliminar duplicados en una sola columna o en un set de columnas.

In [26]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [27]:
data['k3'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,k3
0,one,1,0
3,two,3,3


In [28]:
data.drop_duplicates(['k1', 'k2'])

,k1,k2,k3
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


### Mapear y transformar los datos
A partir de un diccionario, se puede crear una nueva columna para un Dataframe donde las claves del mismo se vinculen con una de las series y los valores formen parte de la nueva columna.

In [29]:
data = pd.DataFrame({'platos': ['panceta', 'bondiola', 'panceta', 'Pastrami',
                           'pavita', 'Panceta', 'pastrami', 'jamon crudo',
                           'nova lox'],
                  'peso': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,platos,peso
0,panceta,4.0
1,bondiola,3.0
2,panceta,12.0
3,Pastrami,6.0
4,pavita,7.5
5,Panceta,8.0
6,pastrami,3.0
7,jamon crudo,5.0
8,nova lox,6.0


In [30]:
data.platos.unique()

array(['panceta', 'bondiola', 'Pastrami', 'pavita', 'Panceta', 'pastrami',
       'jamon crudo', 'nova lox'], dtype=object)

* La idea es ahora poder asignar a cada `plato` un determinado `animal`. Una opción es hacerlo con los métodos `.map()` o `.apply()`.

Repaso:

    1)  pd.DataFrame.apply: Opera sobre filas o columnas completas
    2)  pd.DataFrame.applymap: Opera sobre cada uno de los elementos del Dataframe
    3)  pd.Series.apply: Opera sobre cada uno de los elementos de la Serie. 
    4)  pd.Series.map: Opera sobre cada uno de los elementos de la Serie, muy similar a Series.apply. 

In [31]:
plato_a_animal = {
  'panceta': 'cerdo',
  'bondiola': 'cerdo',
  'pastrami': 'vaca',
  'pavita': 'pavo',
  'jamon crudo': 'cerdo',
  'nova lox': 'salmon'
}

In [32]:
#priemro convertimos a lower y despues mapea cuando encuentra el equivalente y lo pone en una columna
data['platos'] = data['platos'].map(str.lower)
data['animal'] = data['platos'].map(plato_a_animal)
data

,platos,peso,animal
0,panceta,4.0,cerdo
1,bondiola,3.0,cerdo
2,panceta,12.0,cerdo
3,pastrami,6.0,vaca
4,pavita,7.5,pavo
5,panceta,8.0,cerdo
6,pastrami,3.0,vaca
7,jamon crudo,5.0,cerdo
8,nova lox,6.0,salmon


* Ahora quisiéramos poder unificar el formato de los strings y escirbirlos todos en minúsuculas...

In [34]:
data['platos'].map(lambda x: plato_a_animal[x.lower()])

0     cerdo
1     cerdo
2     cerdo
3      vaca
4      pavo
5     cerdo
6      vaca
7     cerdo
8    salmon
Name: platos, dtype: object

### Reemplazar valores
El método data.replace() ofrece varias formas de efectuar reemplazos sobre una serie de Pandas:
    1- Un valor viejo por un valor nuevo.
    2- Una lista de valores viejos por un valor nuevo.
    3- Una lista de valores viejos por una lista de valores nuevos.
    4- Un diccionario que mapee valores nuevos y viejos.

In [35]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [36]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [37]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

* Podemos hacer `replace` diferentes usando una lista de listas...

In [39]:
data.replace([-999, -1000], [0, np.nan])

0    1.0
1    0.0
2    2.0
3    0.0
4    NaN
5    3.0
dtype: float64

* ... O usando un `dict` 

In [40]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renombrar índices de los ejes
Recordemos que los objetos de tipo Index se comportan como las tuplas: son inmutable. Por esta razón no puedo modificar directamente uno de sus elementos como sí puedo hacer con los valores de los Dataframe y Series.

¿Cómo hacemos entonces para cambiar el nombre de una única columna?

El método rename() acepta un diccionario para cada eje donde las keys son los valores viejos y los values son los valores nuevos.

In [41]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Buenos Aires', 'Cordoba', 'Mendoza'],
                 columns=['uno', 'dos', 'tres', 'cuatro'])
data

,uno,dos,tres,cuatro
Buenos Aires,0,1,2,3
Cordoba,4,5,6,7
Mendoza,8,9,10,11


In [42]:
data.index = data.index.map(str.upper)
data

,uno,dos,tres,cuatro
BUENOS AIRES,0,1,2,3
CORDOBA,4,5,6,7
MENDOZA,8,9,10,11


In [43]:
data.rename(index=str.title, columns=str.upper)

# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.title.html

,UNO,DOS,TRES,CUATRO
Buenos Aires,0,1,2,3
Cordoba,4,5,6,7
Mendoza,8,9,10,11


In [44]:
data

,uno,dos,tres,cuatro
BUENOS AIRES,0,1,2,3
CORDOBA,4,5,6,7
MENDOZA,8,9,10,11


In [45]:
data.rename(index={'CORDOBA': 'SANTA FE'},
            columns={'tres': 'ocho'})

,uno,dos,ocho,cuatro
BUENOS AIRES,0,1,2,3
SANTA FE,4,5,6,7
MENDOZA,8,9,10,11


In [46]:
data

,uno,dos,tres,cuatro
BUENOS AIRES,0,1,2,3
CORDOBA,4,5,6,7
MENDOZA,8,9,10,11


In [47]:
# Siempre devuelve una referencia al DataFrame, aunque no quiera utilizarla. Notar el nombre que se le asigna.
#_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

data.rename(index={'CORDOBA': 'SANTA FE'}, inplace=True)
data

,uno,dos,tres,cuatro
BUENOS AIRES,0,1,2,3
SANTA FE,4,5,6,7
MENDOZA,8,9,10,11


### Discretizar y binarizar variables
El proceso de transformar una variable numérica en categórica se llama discretización. 

In [48]:
ages = [26, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

* El método `cut` devuelve el intervalo semi-cerrado al que pertenece cada entrada

In [49]:
# Defino los valores de corte
bins = [18, 25, 35, 60, 100]

# Obtengo una lista de intervalos
# va ah hacer de donde cae cada edad en base a los intervalos de los bins

cats = pd.cut(ages, bins)
cats

[(25, 35], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [50]:
type(cats)

pandas.core.arrays.categorical.Categorical

* El atributo `codes` representa el indice en la lista 'cats'  del intervalo al que pertenece cada entrada

In [51]:
cats.codes
#muestra el codigo de los intervalos

array([1, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [52]:
pd.value_counts(cats)

(25, 35]     4
(18, 25]     4
(35, 60]     3
(60, 100]    1
dtype: int64

In [53]:
pd.value_counts(cats.codes)

1    4
0    4
2    3
3    1
dtype: int64

In [54]:
# Podemos modificar la inclusión del valor de corte en los intervalos
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[26, 36), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [55]:
# Podemos asignar etiquetas a las categorías
group_names = ['Joven', 'Joven Adulto', 'Adulto', 'Senior']
cats.categories = group_names
cats

[Joven Adulto, Joven, Joven, Joven Adulto, Joven, ..., Joven Adulto, Senior, Adulto, Adulto, Joven Adulto]
Length: 12
Categories (4, object): [Joven < Joven Adulto < Adulto < Senior]

In [56]:
cats.value_counts()

Joven           4
Joven Adulto    4
Adulto          3
Senior          1
dtype: int64

* También es posible asignar nombres (etiquetas) a los intervalos generados. Puede hacerse a partir del parámetro `labels=`

In [57]:
pd.cut(ages, [18, 26, 36, 61, 100], labels=['Joven', 'Joven Adulto', 'Adulto', 'Senior'])

[Joven, Joven, Joven, Joven Adulto, Joven, ..., Joven Adulto, Adulto, Adulto, Adulto, Joven Adulto]
Length: 12
Categories (4, object): [Joven < Joven Adulto < Adulto < Senior]

* Qué sucede con el órden de las etiquetas?

In [58]:
serie_ages = pd.cut(ages, bins, labels=group_names)
serie_ages.value_counts()

Joven           4
Joven Adulto    4
Adulto          3
Senior          1
dtype: int64

### Cuantiles en lugar de intervalos preestablecidos

In [59]:
# Divido en cuantiles, en este caso 10
data = np.random.randn(1000)
qcats = pd.qcut(data, 5) 
qcats

[(-2.8, -0.789], (0.212, 0.841], (0.841, 3.74], (0.212, 0.841], (0.841, 3.74], ..., (0.212, 0.841], (0.841, 3.74], (0.841, 3.74], (-0.18, 0.212], (-0.789, -0.18]]
Length: 1000
Categories (5, interval[float64]): [(-2.8, -0.789] < (-0.789, -0.18] < (-0.18, 0.212] < (0.212, 0.841] < (0.841, 3.74]]

In [60]:
qcats.value_counts()

(-2.8, -0.789]     200
(-0.789, -0.18]    200
(-0.18, 0.212]     200
(0.212, 0.841]     200
(0.841, 3.74]      200
dtype: int64

### Detectar y filtrar outliers
No existe un criterio que sea válido en todos los casos para identificar los outliers. El criterio de mayor que el tercer cuartil más 1.5 veces el rango intercuartil o menor que el primer cuartil menos 1.5 veces el rango intercuartil surge de la distribución normal. En esa distribución el 99.7% de la población se encuentra en el rango definido por la media (poblacional) más menos 3 veces el desvío estándar (poblacional)


In [61]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.sample(5)

,0,1,2,3
565,1.502867,-1.400992,1.105131,-1.123506
193,2.666744,1.451456,0.634629,-0.502827
333,0.718409,-1.929776,2.527939,0.731370
767,-0.017749,-1.595617,-1.387784,0.327248
576,-0.442164,1.284845,0.801260,1.025196


In [62]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [63]:
col = data[3]
col[np.abs(col) > 3]
#en este caso la media poblacional es 0 y el desvío es 1 entonces el criterio mencionado anteriormente 
# marcaría como outliers a los valores mayores que 3 o menores que -3
#col[np.abs(col) > 3 * np.std(col)]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [65]:
# Listamos aquellos que no son outliers
data[~(np.abs(data) > 3).any(axis=1)].head()

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570


In [68]:
len(data[~(np.abs(data) > 3).any(axis=1)])

989

In [69]:
# Listamos las filas que tienen eleementos que están en los extremos de la distribución
data[(np.abs(data) > 3).any(1)]

#RAMa con que un valor sea mayor a 3 trae la serie

# data.apply(lambda col:col[np.abs(col) > 3 * np.std(col)])

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [70]:
# Acota el rango de la muestra
# Convierte los valores extremos a esos puntos de referencia
 
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### PARTE II: Variables categóricas y Dummies
Pandas cuenta con el método pd.get_dummies() que recibe una Serie o una lista de Series y realiza el one hot encoding.

Recordemos que una variable con k categorías se puede representar con k-1 variables.

Por eso un parámetro clave de pd.get_dummies es drop_first = True que genera k-1 categorías en lugar de k.

In [71]:
df = pd.DataFrame({'cat_producto': ['b', 'b', 'a', 'c', 'a', 'b'],
                'cod_venta': np.arange(100, 112, 2)})
df

,cat_producto,cod_venta
0,b,100
1,b,102
2,a,104
3,c,106
4,a,108
5,b,110


In [72]:
pd.get_dummies(df['cat_producto'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [73]:
# Agregamos un prefijo para identificar la categoría
pd.get_dummies(df['cat_producto'], prefix='cat_producto')
                         #drop_first=True)


,cat_producto_a,cat_producto_b,cat_producto_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [74]:
dummies = pd.get_dummies(df['cat_producto'], prefix='cat_producto',
                         drop_first=True)
dummies

,cat_producto_b,cat_producto_c
0,1,0
1,1,0
2,0,0
3,0,1
4,0,0
5,1,0


In [75]:
# Concatenamos la columna cod_venta
df_with_dummy = df[['cod_venta']].join(dummies)
df_with_dummy

,cod_venta,cat_producto_b,cat_producto_c
0,100,1,0
1,102,1,0
2,104,0,0
3,106,0,1
4,108,0,0
5,110,1,0


## Manipulación de strings

### String object methods

* `split()` toma un string, lo divide en función de un delimitador (`sep`) y devuelve una lista

In [76]:
val = 'a,b,  guido, asjd, kle'
val.split(',')

['a', 'b', '  guido', ' asjd', ' kle']

* `strip()` toma un string y devuelve un string sin los espacios iniciales y finales.

In [77]:
# Ejemplos:

texto = "   Este es el primer ejemplo....wow!!!   ";
print(texto.strip())

texto1 = "0000000Este es el segundo ejemplo....wow!!!0000000";
print(texto1.strip( '0' ))

Este es el primer ejemplo....wow!!!
Este es el segundo ejemplo....wow!!!


In [78]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido', 'asjd', 'kle']

* `find()` devuelve el índice más bajo dentro de un string en el cual un substring es encontrado. Devuelve -1 si no la encuentra

In [79]:
val.find(':')

-1

In [81]:
val.find('b')
#devuelve el indice

2

* `index()` es similar, pero devuelve un `ValueError` cuando no encuentra el substring buscado

In [82]:
val.index(',')

1

In [83]:
# Genera un error si no encuentra el substring
val.index(':')

ValueError: substring not found

In [84]:
# El mismo ejemplo de arriba pero
# usando excepciones.
# Nos permiten manejar errores en tiempo de ejecución

try:
    val.index(':')
except:
    print("Error, substring no encontrado!")

Error, substring no encontrado!


* `count()` cuenta la ocurrencia de un substring determinado en un string mayor.

In [85]:
val.count(',')

4

* `replace()` reemplaza un substring por otro.

In [86]:
val.replace(',', ';')

'a;b;  guido; asjd; kle'

### Funciones vectorizadas para strings en Pandas

In [87]:
import re
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)

In [88]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [89]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [90]:
pattern = "\w+"

In [94]:
data.str.findall(pattern, flags=re.IGNORECASE)[0][-1]

'com'

In [95]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

### Ejemplo: Dataset movies

In [96]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.csv', header=None,
                        names=mnames, encoding="latin9", sep=';')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [97]:
lista = [sublista.split('|') for sublista in movies.genres]
lista[:10]

[['Animation', "Children's", 'Comedy'],
 ['Adventure', "Children's", 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama'],
 ['Comedy'],
 ['Action', 'Crime', 'Thriller'],
 ['Comedy', 'Romance'],
 ['Adventure', "Children's"],
 ['Action'],
 ['Action', 'Adventure', 'Thriller']]

In [98]:
# Aplanamos la lista anterior
genres = sorted(set([item for s in lista for item in s]))
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [99]:
# Creamos un DataFrame vacío para asociar los géneros correspondientes a cada película
dummies = pd.DataFrame(np.zeros((len(movies), len(genres)), dtype=int), columns=genres)
dummies.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [100]:
# Codifica las categorías como dummies. Escribe un 1 donde corresponde
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')] = 1

In [101]:
dummies.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [102]:
movies_final = movies.join(dummies.add_prefix('Genre_'))
movies_final.head()

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
